## Imports

In [10]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score

import sys
import pickle
import wandb
import yaml

import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.utils import to_networkx
import torch.nn as nn
from torch.nn import Sequential, Linear
import networkx as nx

from utils.miscellaneous import read_config
from utils.miscellaneous import create_folder_structure_MLPvsGNN
from utils.miscellaneous import initalize_random_generators

from training.train import training
from training.test import testing

from utils.visualization import plot_R2, plot_loss

### Parse configuration file + initializations


In [11]:
# read config files
cfg = read_config("config_unrolling.yaml")
# create folder for results
exp_name = cfg['exp_name']
data_folder = cfg['data_folder']
results_folder = create_folder_structure_MLPvsGNN(cfg, parent_folder='./experiments')


all_wdn_names = cfg['networks']
initalize_random_generators(cfg, count=0)

# initialize pytorch device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#torch.set_num_threads(12)

Creating folder: ./experiments/unrolling_WDN
cuda:0


In [12]:
# TO DO: at the moment I am not using the parsed values for batch size and num_epochs ;
# I am not using alpha as well because the loss has no "smoothness" penalty (yet)
batch_size = cfg['trainParams']['batch_size']
alpha = cfg['lossParams']['alpha']
res_columns = ['train_loss', 'valid_loss','test_loss','max_train_loss', 'max_valid_loss','max_test_loss', 'min_train_loss', 'min_valid_loss','min_test_loss','r2_train', 'r2_valid',
			   'r2_test','total_params','total_time','test_time']

# Functions

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator,TransformerMixin

class PowerLogTransformer(BaseEstimator,TransformerMixin):
	def __init__(self,log_transform=False,power=4,reverse=True):
		if log_transform == True:
			self.log_transform = log_transform
			self.power = None
		else:
			self.power = power
			self.log_transform = None
		self.reverse=reverse
		self.max_ = None
		self.min_ = None

	def fit(self,X,y=None):
		self.max_ = np.max(X)
		self.min_ = np.min(X)
		return self

	def transform(self,X):
		if self.log_transform==True:
			if self.reverse == True:
				return np.log1p(self.max_-X)
			else:
				return np.log1p(X-self.min_)
		else:
			if self.reverse == True:
				return (self.max_-X)**(1/self.power )
			else:
				return (X-self.min_)**(1/self.power )

	def inverse_transform(self,X):
		if self.log_transform==True:
			if self.reverse == True:
				return (self.max_ - np.exp(X))
			else:
				return (np.exp(X) + self.min_)
		else:
			if self.reverse == True:
				return (self.max_ - X**self.power )
			else:
				return (X**self.power + self.min_)


class GraphNormalizer:
	def __init__(self, x_feat_names=['elevation', 'base_demand', 'base_head'],
				 ea_feat_names=['diameter', 'length', 'roughness'], output='head'):
		# store
		self.x_feat_names = x_feat_names
		self.ea_feat_names = ea_feat_names
		self.output = output

		# create separate scaler for each feature (can be improved, e.g., you can fit a scaler for multiple columns)
		self.scalers = {}
		for feat in self.x_feat_names:
			if feat == 'elevation':
				self.scalers[feat] = PowerLogTransformer(log_transform=True, reverse=False)
			else:
				self.scalers[feat] = MinMaxScaler()
		self.scalers[output] = PowerLogTransformer(log_transform=True, reverse=True)
		for feat in self.ea_feat_names:
			if feat == 'length':
				self.scalers[feat] = PowerLogTransformer(log_transform=True, reverse=False)
			else:
				self.scalers[feat] = MinMaxScaler()

	def fit(self, graphs):
		''' Fit the scalers on an array of x and ea features
        '''
		x, y, ea = from_graphs_to_pandas(graphs)
		for ix, feat in enumerate(self.x_feat_names):
			self.scalers[feat] = self.scalers[feat].fit(x[:, ix].reshape(-1, 1))
		self.scalers[self.output] = self.scalers[self.output].fit(y.reshape(-1, 1))
		for ix, feat in enumerate(self.ea_feat_names):
			self.scalers[feat] = self.scalers[feat].fit(ea[:, ix].reshape(-1, 1))
		return self

	def transform(self, graph):
		''' Transform graph based on normalizer
        '''
		graph = graph.clone()
		for ix, feat in enumerate(self.x_feat_names):
			temp = graph.x[:, ix].numpy().reshape(-1, 1)
			graph.x[:, ix] = torch.tensor(self.scalers[feat].transform(temp).reshape(-1))
		for ix, feat in enumerate(self.ea_feat_names):
			temp = graph.edge_attr[:, ix].numpy().reshape(-1, 1)
			graph.edge_attr[:, ix] = torch.tensor(self.scalers[feat].transform(temp).reshape(-1))
		graph.y = torch.tensor(self.scalers[self.output].transform(graph.y.numpy().reshape(-1, 1)).reshape(-1))
		return graph

	def inverse_transform(self, graph):
		''' Perform inverse transformation to return original features
        '''
		graph = graph.clone()
		for ix, feat in enumerate(self.x_feat_names):
			temp = graph.x[:, ix].numpy().reshape(-1, 1)
			graph.x[:, ix] = torch.tensor(self.scalers[feat].inverse_transform(temp).reshape(-1))
		for ix, feat in enumerate(self.ea_feat_names):
			temp = graph.edge_attr[:, ix].numpy().reshape(-1, 1)
			graph.edge_attr[:, ix] = torch.tensor(self.scalers[feat].inverse_transform(temp).reshape(-1))
		graph.y = torch.tensor(self.scalers[self.output].inverse_transform(graph.y.numpy().reshape(-1, 1)).reshape(-1))
		return graph

	def transform_array(self, z, feat_name):
		'''
            This is for MLP dataset; it can be done better (the entire thing, from raw data to datasets)
        '''
		return torch.tensor(self.scalers[feat_name].transform(z).reshape(-1))

	def inverse_transform_array(self, z, feat_name):
		'''
            This is for MLP dataset; it can be done better (the entire thing, from raw data to datasets)
        '''
		return torch.tensor(self.scalers[feat_name].inverse_transform(z).reshape(-1))

def from_graphs_to_pandas(graphs, l_x=3, l_ea=3):
	x = []
	y = []
	ea = []
	for i, graph in enumerate(graphs):
		x.append(graph.x.numpy())
		y.append(graph.y.reshape(-1, 1).numpy())
		ea.append(graph.edge_attr.numpy())
	return np.concatenate(x, axis=0), np.concatenate(y, axis=0), np.concatenate(ea, axis=0)


In [14]:
# constant indexes for node and edge features
HEAD_INDEX = 0
BASEDEMAND_INDEX = 1
TYPE_INDEX = 2
DIAMETER_INDEX = 0
LENGTH_INDEX = 1
ROUGHNESS_INDEX = 2
FLOW_INDEX = 3

def load_raw_dataset(wdn_name, data_folder):
	'''
	Load tra/val/data for a water distribution network datasets
	-------
	wdn_name : string
		prefix of pickle files to open
	data_folder : string
		path to datasets
	'''

	data_tra = pickle.load(open(f'{data_folder}/train/{wdn_name}.p', "rb"))
	data_val = pickle.load(open(f'{data_folder}/valid/{wdn_name}.p', "rb"))
	data_tst = pickle.load(open(f'{data_folder}/test/{wdn_name}.p', "rb"))

	return data_tra, data_val, data_tst

def create_dataset(database, normalizer=None, HW_rough_minmax=[60, 150]):
	'''
	Creates working datasets dataset from the pickle databases
	------
	database : list
		each element in the list is a pickle file containing Data objects
	normalization: dict
		normalize the dataset using mean and std
	'''
	# Roughness info (Hazen-Williams) / TODO: remove the hard_coding
	minR = HW_rough_minmax[0]
	maxR = HW_rough_minmax[1]

	graphs = []

	for i in database:
		graph = torch_geometric.data.Data()

		# Node attributes
		# elevation_head = i.elevation + i.base_head
		# elevation_head = i.elevation.clone()
		# elevation_head[elevation_head == 0] = elevation_head.mean()

		min_elevation = min(i.elevation[i.type_1H == 0])
		head = i.pressure + i.base_head + i.elevation
		# elevation_head[i.type_1H == 1] = head[i.type_1H == 1]
		# elevation = elevation_head - min_elevation

		# base_demand = i.base_demand * 1000  # convert to l/s
		# graph.x = torch.stack((i.elevation, i.base_demand, i.type_1H*i.base_head), dim=1).float()
		graph.x = torch.stack((i.elevation+i.base_head, i.base_demand, i.type_1H), dim=1).float()
		# graph.x = torch.stack((i.elevation+i.base_head, i.base_demand, i.type_1H), dim=1).float()

		# Position and ID
		graph.pos = i.pos
		graph.ID = i.ID

		# Edge index (Adjacency matrix)
		graph.edge_index = i.edge_index

		# Edge attributes
		diameter = i.diameter
		length = i.length
		roughness = i.roughness
		graph.edge_attr = torch.stack((diameter, length, roughness), dim=1).float()

		# Graph output (head)
		graph.y = head[i.type_1H == 0].reshape(-1, 1)

		# normalization
		if normalizer is not None:
			graph = normalizer.transform(graph)

		graphs.append(graph)
	A12 = nx.incidence_matrix(to_networkx(graphs[0]), oriented=True).toarray().transpose()
	return graphs, A12

def create_dataset_MLP_from_graphs(graphs, features=['nodal_demands', 'base_heads','diameter','roughness','length'],no_res_out=True):

	# index edges to avoid duplicates: this considers all graphs to be UNDIRECTED!
	ix_edge = graphs[0].edge_index.numpy().T
	ix_edge = (ix_edge[:, 0] < ix_edge[:, 1])

	# position of reservoirs
	ix_res = graphs[0].x[:,TYPE_INDEX].numpy()>0
	indices = []
	for ix_feat, feature in enumerate(features):
		for ix_item, item in enumerate(graphs):
			if feature == 'diameter':
				x_ = item.edge_attr[ix_edge,DIAMETER_INDEX]
			elif feature == 'roughness':
				# remove reservoirs
				x_ = item.edge_attr[ix_edge,ROUGHNESS_INDEX]
			elif feature == 'length':
				# remove reservoirs
				x_ = item.edge_attr[ix_edge,LENGTH_INDEX]
			elif feature == 'nodal_demands':
				# remove reservoirs
				x_ = item.x[~ix_res,BASEDEMAND_INDEX]
			elif feature == 'base_heads':
				x_ = item.x[ix_res,HEAD_INDEX]
			else:
				raise ValueError(f'Feature {feature} not supported.')
			if ix_item == 0:
				x = x_
			else:
				x = torch.cat((x, x_), dim=0)
		if ix_feat == 0:
			X = x.reshape(len(graphs), -1)
		else:
			X = torch.cat((X, x.reshape(len(graphs), -1)), dim=1)
		indices.append(X.shape[1])
	for ix_item, item in enumerate(graphs):
		# remove reservoirs from y as well
		if ix_item == 0:
			if no_res_out == True:
				y = item.y
			else:
				y = item.y[~ix_res]
		else:
			if no_res_out == True:
				y = torch.cat((y, item.y), dim=0)
			else:
				y = torch.cat((y, item.y[~ix_res]), dim=0)
	y = y.reshape(len(graphs), -1)

	return torch.utils.data.TensorDataset(X, y), X.shape[1], indices

def create_incidence_matrices(graphs,incidence_matrix):

	# position of reservoirs

	ix_res = graphs[0].x[:,TYPE_INDEX].cpu().numpy()>0
	ix_edge = graphs[0].edge_index.numpy().T
	ix_edge = (ix_edge[:, 0] < ix_edge[:, 1])
	incidence_matrix = incidence_matrix[ix_edge,:]
	A10 = incidence_matrix[:, ix_res]
	A12 = incidence_matrix[:, ~ix_res]
	A12[np.where(A10 == 1),:] *= -1
	A10[np.where(A10 == 1),:] *= -1
	return A10, A12

## Models
I will be Creating different models as follows:

* A simple MLP
* An unrolled version of Heads and Flows, without static variables
* An unrolled version with Heads, Flows and static variables


In [15]:
class MLP(nn.Module):
	def __init__(self, num_outputs, hid_channels, indices, num_layers=6):
		super(MLP, self).__init__()
		torch.manual_seed(42)
		self.hid_channels = hid_channels
		self.indices = indices
		self.num_heads = indices[0]
		self.num_flows = indices[2] - indices[1]

		layers = [Linear(indices[4], hid_channels),
				  nn.ReLU()]

		for l in range(num_layers-1):
			layers += [Linear(hid_channels, hid_channels),
					   nn.ReLU()]

		layers += [Linear(hid_channels, num_outputs)]

		self.main = nn.Sequential(*layers)

	def forward(self, x):

		x = self.main(x)

		return x

In [16]:
class BaselineUnrolling(nn.Module):
	def __init__(self,num_outputs, indices, num_blocks):
		super(BaselineUnrolling, self).__init__()
		torch.manual_seed(42)
		self.indices = indices
		self.num_heads = indices[0]
		self.num_flows = indices[2]-indices[1]
		self.num_blocks = num_blocks

		self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

		self.hid_HF = nn.ModuleList()
		self.hid_FH = nn.ModuleList()

		for i in range(num_blocks):
			self.hid_HF.append(Sequential(Linear(self.num_heads,self.num_flows), nn.ReLU()))
			self.hid_FH.append(Sequential(Linear(self.num_flows, self.num_heads),
						   nn.ReLU()))
		self.out = Linear(self.num_flows, num_outputs)

	def forward(self, x):

		d = x[:,self.indices[1]:self.indices[2]].double().view(-1,self.num_flows,1)

		q =  torch.mul(math.pi/4, torch.pow(d,2)).view(-1,self.num_flows)

		for i in range(self.num_blocks-1):
			h = self.hid_FH[i](q)
			q = q - self.hid_HF[i](h)

		return self.out(q)

In [17]:
class UnrollingModel(nn.Module):
	def __init__(self, num_outputs, indices, num_blocks):
		super(UnrollingModel, self).__init__()
		torch.manual_seed(42)
		self.indices = indices
		self.num_heads = indices[0]
		self.num_flows = indices[2]-indices[1]
		self.num_blocks = num_blocks

		self.hidq0_h = Linear(indices[2]-indices[1], self.num_heads)
		self.hidh0_q = Linear(indices[1]-indices[0], self.num_flows)
		self.hidh0_h = Linear(indices[1]-indices[0], self.num_heads)
		self.hids_q =  Linear(indices[0], self.num_flows)
		self.hid_S = Sequential(Linear(indices[4] - indices[1], self.num_flows),
						   nn.ReLU())

		self.hid_hf = nn.ModuleList()
		self.hid_fh = nn.ModuleList()
		self.resq = nn.ModuleList()
		self.hidD_q = nn.ModuleList()
		self.hidD_h = nn.ModuleList()

		for i in range(num_blocks):
			self.hid_hf.append(Sequential(Linear(self.num_heads,self.num_flows), nn.PReLU()))
			self.hid_fh.append(Sequential(Linear(self.num_flows, self.num_heads),
						   nn.ReLU()))
			self.resq.append(Sequential(Linear(self.num_flows,self.num_heads),
						   nn.PReLU()))
			self.hidD_q.append(Sequential(Linear(self.num_flows,self.num_flows), nn.PReLU()))
			self.hidD_h.append(Sequential(Linear(self.num_flows,self.num_heads),
							   nn.ReLU()))

		self.out = Linear(self.num_flows, num_outputs)

	def forward(self, x):

		s, h0, d, edge_features = x[:,:self.indices[0]], x[:,self.indices[0]:self.indices[1]], x[:,self.indices[1]:self.indices[2]], x[:,self.indices[1]:]
		res_h0_q, res_s_q, res_h0_h, res_S_q = self.hidh0_q(h0), self.hids_q(s), self.hidh0_h(h0), self.hid_S(edge_features)

		q =  torch.mul(math.pi/4, torch.pow(d,2))
		res_q_h = self.hidq0_h(q)

		for i in range(self.num_blocks-1):

			D_q = self.hidD_q[i](torch.mul(q, res_S_q))
			D_h = self.hidD_h[i](D_q)
			hid_x = torch.mul(D_q,torch.sum(torch.stack([q, res_s_q, res_h0_q]),dim=0))
			h = self.hid_fh[i](hid_x)
			hid_x = self.hid_hf[i](torch.mul(torch.sum(torch.stack([h,res_h0_h,res_q_h]),dim=0), D_h))
			q = torch.sub(q,hid_x)
			res_q_h = self.resq[i](q)

		return h

## Running experiments

In [18]:
for ix_wdn, wdn in enumerate(all_wdn_names):
	print(f'\nWorking with {wdn}, network {ix_wdn+1} of {len(all_wdn_names)}')

	# retrieve wntr data
	tra_database, val_database, tst_database = load_raw_dataset(wdn, data_folder)
	# reduce training data
	# tra_database = tra_database[:int(len(tra_database)*cfg['tra_prc'])]
	if cfg['tra_num'] < len(tra_database):
		tra_database = tra_database[:cfg['tra_num']]

	# remove PES anomaly
	if wdn == 'PES':
		if len(tra_database)>4468:
			del tra_database[4468]
			print('Removed PES anomaly')
			print('Check',tra_database[4468].pressure.mean())

	# get GRAPH datasets    # later on we should change this and use normal scalers from scikit
	tra_dataset, A12_bar = create_dataset(tra_database)
	gn = GraphNormalizer()
	gn = gn.fit(tra_dataset)
	tra_dataset, _ = create_dataset(tra_database,normalizer=gn)
	val_dataset,_ = create_dataset(val_database,normalizer=gn)
	tst_dataset,_ = create_dataset(tst_database,normalizer=gn)
	node_size, edge_size = tra_dataset[0].x.size(-1), tra_dataset[0].edge_attr.size(-1)
	# number of nodes
	# n_nodes=tra_dataset[0].x.shape[0]
	n_nodes=(1-tra_database[0].type_1H).numpy().sum() # remove reservoirs
	# dataloader
	# transform dataset for MLP
	# We begin with the MLP versions, when I want to add GNNs, check Riccardo's code
	A10,A12 = create_incidence_matrices(tra_dataset, A12_bar)
	tra_dataset_MLP, num_inputs, indices = create_dataset_MLP_from_graphs(tra_dataset)
	val_dataset_MLP = create_dataset_MLP_from_graphs(val_dataset)[0]
	tst_dataset_MLP = create_dataset_MLP_from_graphs(tst_dataset)[0]
	tra_loader = torch.utils.data.DataLoader(tra_dataset_MLP,
											 batch_size=batch_size, shuffle=True)
	val_loader = torch.utils.data.DataLoader(val_dataset_MLP,
											 batch_size=batch_size, shuffle=False)
	tst_loader = torch.utils.data.DataLoader(tst_dataset_MLP,
											 batch_size=batch_size, shuffle=False)
	# loop through different algorithms
	n_epochs = num_inputs
	for algorithm in cfg['algorithms']:

		hyperParams = cfg['hyperParams'][algorithm]
		all_combinations = ParameterGrid(hyperParams)

		# create results dataframe
		results_df = pd.DataFrame(list(all_combinations))
		results_df = pd.concat([results_df,
								pd.DataFrame(index=np.arange(len(all_combinations)),
										  columns=list(res_columns))],axis=1)

		for i, combination in enumerate(all_combinations):
			print(f'{algorithm}: training combination {i+1} of {len(all_combinations)}\n')
			combination['indices'] = indices
			combination['num_outputs'] = n_nodes
			if algorithm == 'UnrollingMLP' or algorithm == 'BaselineUnrolling':
				combination['A12'] = A12
				combination['A10'] = A10

			wandb.config = combination

			# model creation
			model = getattr(sys.modules[__name__], algorithm)(**combination).double().to(device)
			# print(model)
			total_parameters = sum(p.numel() for p in model.parameters())

			# model optimizer
			optimizer = optim.Adam(params=model.parameters(), **cfg['adamParams'])

			# training
			model, tra_losses, val_losses, elapsed_time = training(model, optimizer, tra_loader, val_loader,
																	patience=10, report_freq=20, n_epochs=n_epochs,
																   alpha=alpha, lr_rate=2, lr_epoch=20,
																   normalization=None, path = f'{results_folder}/{wdn}/{algorithm}/')
			plot_loss(tra_losses,val_losses,f'{results_folder}/{wdn}/{algorithm}/loss/{i}')
			plot_R2(model,val_loader,f'{results_folder}/{wdn}/{algorithm}/R2/{i}', normalization=gn)
			# store training history and model
			pd.DataFrame(data = np.array([tra_losses, val_losses]).T).to_csv(
				f'{results_folder}/{wdn}/{algorithm}/hist/{i}.csv')
			torch.save(model, f'{results_folder}/{wdn}/{algorithm}/models/{i}.csv')

			# compute and store predictions, compute r2 scores
			losses = {}
			max_losses = {}
			min_losses = {}
			r2_scores = {}
			for split, loader in zip(['training','validation','testing'],[tra_loader,val_loader,tst_loader]):
				losses[split], max_losses[split], min_losses[split], pred, real,test_time = testing(model, loader, normalization=gn)
				r2_scores[split] = r2_score(real, pred)
				if i == 0:
					pd.DataFrame(data=real.reshape(-1,n_nodes)).to_csv(
						f'{results_folder}/{wdn}/{algorithm}/pred/{split}/real.csv') # save real obs
				pd.DataFrame(data=pred.reshape(-1,n_nodes)).to_csv(
					f'{results_folder}/{wdn}/{algorithm}/pred/{split}/{i}.csv')

			# store results
			results_df.loc[i,res_columns] = (losses['training'], losses['validation'], losses['testing'],
											 max_losses['training'], max_losses['validation'], max_losses['testing'],
											 min_losses['training'], min_losses['validation'], min_losses['testing'],
											 r2_scores['training'], r2_scores['validation'], r2_scores['testing'],
											 total_parameters, elapsed_time,test_time)
		# # save graph normalizer
		with open(f'{results_folder}/{wdn}/{algorithm}/gn.pickle', 'wb') as handle:
		     pickle.dump(gn, handle, protocol=pickle.HIGHEST_PROTOCOL)

		with open(f'{results_folder}/{wdn}/{algorithm}/model.pickle', 'wb') as handle:
			torch.save(model, handle)
		results_df.to_csv(f'{results_folder}/{wdn}/{algorithm}/results_{algorithm}.csv')


Working with FOS, network 1 of 5


C:\Users\alber\AppData\Local\Temp\ipykernel_14048\3711682033.py:80: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A12 = nx.incidence_matrix(to_networkx(graphs[0]), oriented=True).toarray().transpose()
C:\Users\alber\AppData\Local\Temp\ipykernel_14048\3711682033.py:80: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A12 = nx.incidence_matrix(to_networkx(graphs[0]), oriented=True).toarray().transpose()
C:\Users\alber\AppData\Local\Temp\ipykernel_14048\3711682033.py:80: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A12 = nx.incidence_matrix(to_networkx(graphs[0]), oriented=True).toarray().transpose()
C:\Users\alber\AppData\Local\Temp\ipykernel_14048\3711682033.py:80: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A12 = nx.incidence_matrix(to_networkx(graphs[0]), o

UnrollingModel: training combination 1 of 7



  9%|▉         | 20/211 [00:05<00:45,  4.19it/s]

Learning rate is divided by  2 to: 0.0005
epoch: 20 	 train MSE: 0.1284 	 val MSE: 0.1351


...

UnrollingModel: training combination 7 of 7



  1%|          | 20/1806 [01:27<2:02:30,  4.12s/it]

Learning rate is divided by  2 to: 0.0005
epoch: 20 	 train MSE: 0.0466 	 val MSE: 0.0537


  1%|▏         | 24/1806 [01:47<2:13:14,  4.49s/it]

Early Stopping
